In [29]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
sys.path.append("../") # Add parent directory to path

In [30]:
# Check if GPU is available
# if torch.cuda.is_available():
#     device = torch.device("cuda")
#     print("Using GPU:", torch.cuda.get_device_name(0))
device = torch.device("cpu")
print("Using device:", device)

Using device: cpu


In [33]:
# load input data
x_test = np.load("../data/L2Hr/train_input.npy")


In [34]:
zs = [0.,.2, .4, .6, .8, 1.]

In [35]:
x_test.shape

(36, 10)

In [36]:
from gokunemu import MatterPowerEmulator
emu = MatterPowerEmulator(device=device)

In [37]:
# test speed
# a bunch of calls to the emulator

%timeit emu.predict(x_test[0], redshifts=zs)

3.06 ms ± 174 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [39]:
import time

n_calls = 5000  
start = time.time()
for _ in range(n_calls):
    _ = emu.predict(x_test[0], redshifts=[0.,1.1,2.09, 0.33,.44])
end = time.time()

print(f"Average time per call: {(end - start) / n_calls:.6f} seconds")


Average time per call: 0.003218 seconds


In [40]:
# multiple cosmologies

n_calls = 1000  
start = time.time()
for _ in range(n_calls):
    _ = emu.predict(x_test, redshifts=[0.,1.1,2.09, 0.33,.44])
end = time.time()

print(f"Average time per call: {(end - start) / n_calls:.6f} seconds")


Average time per call: 0.012833 seconds
